<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Analytics" data-toc-modified-id="Analytics-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analytics</a></span><ul class="toc-item"><li><span><a href="#Data-set" data-toc-modified-id="Data-set-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data set</a></span></li><li><span><a href="#Question-1" data-toc-modified-id="Question-1-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Question 1</a></span></li><li><span><a href="#Question-2" data-toc-modified-id="Question-2-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Question 2</a></span></li><li><span><a href="#Question-3" data-toc-modified-id="Question-3-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Question 3</a></span></li><li><span><a href="#Question-4" data-toc-modified-id="Question-4-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Question 4</a></span></li></ul></li></ul></div>

# Analytics
## Data set
* It record users' visits of websites
* It contains 4 columns, ts for timestamp, user_id, country_id, site_id
* It contains 3554 rows

In [10]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
# Load libraries
import pandas as pd

# Read the data set
visit_df = pd.read_csv('./Data/Analytics.csv')
visit_df.head()
visit_df.info()

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 4 columns):
ts            3553 non-null object
user_id       3553 non-null object
country_id    3553 non-null object
site_id       3553 non-null object
dtypes: object(4)
memory usage: 111.1+ KB


In [13]:
# Check duplicates
visit_df.duplicated().sum()

0

## Question 1
Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?

To solve this question,
1. Select rows with country_id = "BDV"
2. In this subset, compute the number of unique user_id for each site_id (i.e., group by site_id)

In [14]:
visit_df[visit_df.country_id == 'BDV'].groupby('site_id').nunique()['user_id']

site_id
3POLC      2
5NPAU    544
N0OTG     90
Name: user_id, dtype: int64

The site '5NPAU' has the largest number of unique users. The number is 544.

## Question 2
Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times.

To solve this problem,
1. Subset the records between 2019-02-03 00:00:00 and 2019-02-04 23:59:59
2. In this subset, count the number of visits for each user_id and site_id (i.e., group by user_id and site_id)
3. From the output of the above step, select records which has more than 10 visits

In [15]:
# For the column of ts, cast object type to datetime
visit_df['ts'] = pd.to_datetime(visit_df['ts'])
visit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 4 columns):
ts            3553 non-null datetime64[ns]
user_id       3553 non-null object
country_id    3553 non-null object
site_id       3553 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 111.1+ KB


In [16]:
visit_subset = visit_df[(visit_df.ts >= '2019-02-03 00:00:00')&(visit_df.ts <= '2019-02-04 23:59:59')]

In [17]:
vis_sub_cnt = visit_subset.groupby(['user_id','site_id']).size().to_frame('count').reset_index()

In [18]:
vis_sub_cnt[vis_sub_cnt['count'] > 10]

,user_id,site_id,count
3,LC06C3,N0OTG,25
417,LC3A59,N0OTG,26
485,LC3C7E,3POLC,15
493,LC3C9D,N0OTG,17


Here are the 4 users asked, the website they visited more than 10 times, and the exact number of visits: <br>
('LC06C3',	'N0OTG',	25) <br>
('LC3A59',	'N0OTG',	26) <br>
('LC3C7E',	'3POLC',	15) <br>
('LC3C9D',	'N0OTG',	17)

## Question 3
For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. <br>
Based on this measure, what are top three sites?

To solve this problem,
1. For each user, find the site they visit last
    * Group by user and find the entry which has the max ts for each user
2. Using the output from the above step, group by site and count the unique number of users
3. Output the top 3 sites 

In [19]:
# Find the last time stamp for each user
last_vis1 = visit_df.groupby('user_id').agg({'ts': max}).reset_index()
last_vis1.head()
len(last_vis1)

,user_id,ts
0,LC00C3,2019-02-03 18:52:50
1,LC01C3,2019-02-04 11:35:10
2,LC05C3,2019-02-02 14:14:44
3,LC06C3,2019-02-07 01:16:12
4,LC07C3,2019-02-05 19:06:42


1916

In [20]:
# Find the corresponding site each user last visited
last_vis2 = pd.merge(visit_df, last_vis1, on = ['user_id','ts'])
last_vis2.head()
len(last_vis2)

,ts,user_id,country_id,site_id
0,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
1,2019-02-01 00:21:50,LC3500,TL6,N0OTG
2,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G
3,2019-02-01 00:41:50,LCC3C3,QLT,5NPAU
4,2019-02-01 00:42:13,LC39C8,TL6,QGO3G


1916

In [21]:
# With the output above, count the unique users for each site
last_vis2.groupby('site_id').nunique()['user_id'].sort_values(ascending = False)

site_id
5NPAU    992
N0OTG    561
QGO3G    289
GVOFK     42
3POLC     28
RT9Z6      2
JSUUP      1
EUZ/Q      1
Name: user_id, dtype: int64

Based on this metric, the top 3 sites and their the number of unique users are: <br>
('5NPAU', 992) <br>
('N0OTG', 561) <br>
('QGO3G', 289)

## Question 4
For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first and last visits are to the same website. What is the number?

To solve this problem,
1. For each user, find their first and last visited site
    * Group by user, min. ts to find the first visit time
    * Group by user, max. ts to find the last visit time
2. Find the corresponding sites for each user's first and last visit
3. Count users who visited the same site at the first and last time

In [22]:
# Find the first and last visiting time for each user
first_last = visit_df.groupby('user_id').agg({'ts': [min, max]}).reset_index()
first_last.head()
len(first_last)

user_id                  ts                    
                          min                 max
0  LC00C3 2019-02-03 18:52:50 2019-02-03 18:52:50
1  LC01C3 2019-02-04 11:35:10 2019-02-04 11:35:10
2  LC05C3 2019-02-02 14:14:44 2019-02-02 14:14:44
3  LC06C3 2019-02-01 22:49:39 2019-02-07 01:16:12
4  LC07C3 2019-02-05 19:06:42 2019-02-05 19:06:42

1916

In [23]:
first_last.columns = ['user_id','ts_min','ts_max']

In [24]:
first_last.head()

,user_id,ts_min,ts_max
0,LC00C3,2019-02-03 18:52:50,2019-02-03 18:52:50
1,LC01C3,2019-02-04 11:35:10,2019-02-04 11:35:10
2,LC05C3,2019-02-02 14:14:44,2019-02-02 14:14:44
3,LC06C3,2019-02-01 22:49:39,2019-02-07 01:16:12
4,LC07C3,2019-02-05 19:06:42,2019-02-05 19:06:42


In [26]:
# Find the corresponding sites for each user
first_last2 = pd.merge(visit_df, first_last, left_on = ['user_id','ts'], right_on = ['user_id','ts_min'])
first_last2.head()
len(first_last2)

first_last3 = pd.merge(visit_df, first_last2, left_on = ['user_id','ts'], right_on = ['user_id','ts_max'])
first_last3.tail()
len(first_last3)

,ts,user_id,country_id,site_id,ts_min,ts_max
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG,2019-02-01 00:01:24,2019-02-07 00:24:50
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG,2019-02-01 00:10:19,2019-02-01 00:10:19
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG,2019-02-01 00:21:50,2019-02-01 00:21:50
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG,2019-02-01 00:22:50,2019-02-03 04:50:43
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G,2019-02-01 00:23:44,2019-02-01 00:23:44


1916

,ts_x,user_id,country_id_x,site_id_x,ts_y,country_id_y,site_id_y,ts_min,ts_max
1911,2019-02-07 23:44:34,LC3561,TL6,N0OTG,2019-02-01 00:25:29,TL6,3POLC,2019-02-01 00:25:29,2019-02-07 23:44:34
1912,2019-02-07 23:55:07,LC3837,TL6,RT9Z6,2019-02-03 03:30:25,QLT,5NPAU,2019-02-03 03:30:25,2019-02-07 23:55:07
1913,2019-02-07 23:56:57,LC3F13,TL6,QGO3G,2019-02-07 23:56:57,TL6,QGO3G,2019-02-07 23:56:57,2019-02-07 23:56:57
1914,2019-02-07 23:58:56,LC35EB,TL6,QGO3G,2019-02-07 23:58:56,TL6,QGO3G,2019-02-07 23:58:56,2019-02-07 23:58:56
1915,2019-02-07 23:59:37,LC3842,HVQ,3POLC,2019-02-05 16:21:30,HVQ,3POLC,2019-02-05 16:21:30,2019-02-07 23:59:37


1916

In [27]:
same_site = first_last3[first_last3.site_id_x == first_last3.site_id_y]
len(same_site)

1670

In [28]:
len(first_last[first_last.ts_min == first_last.ts_max])

1261

There are 1670 users who visited the same website when they first and last visited. Out of these users, 1261 users only visited once (ts_min = ts_max), so their first and last visits are the same. <br>
To further analyze, we can count the unique users for each such site (i.e., visited by users the first and last time)